# Mejorando La Aplicacion con Docker Compose

*Docker Compose* es una herramienta para definir y ejecutar aplicaciones Docker multi-contenedor. En lugar de ejecutar cada contenedor individualmente con comandos ``docker run``, *Docker Compose* te permite definir todos tus servicios, redes y volúmenes en un solo archivo YAML (normalmente llamado **docker-compose.yml**). Luego, puedes levantar y bajar toda tu aplicación con un solo comando: ``docker compose up``

Inicialmente el servicio de la FAST API aplicacion se levantaba con *docker run*:

```sh
docker run --rm -it --publish 8000:8000  --name app-temporal --network platzinet fastapi-app-image
```

Asi quedaria el `docker-compose.yml`

```sh
services:
  app-temp:
    build: 
      context: . 
      dockerfile: Dockerfile 👈
    ports:
      - 8000:8000
    volumes:
      - .:/web-server 👈
    networks:
      - platzinet

    
networks:
  platzinet:
    external: True
```
- `build`: Indica a Docker Compose que debe construir la imagen para este servicio. 

- ``volumes:`` Crea un montaje de volumen (**bind mount**). Esto sincroniza el contenido de tu directorio local (.) con el directorio /web-server dentro del contenedor. Ideal para un *hot reload*, cualquier cambio que hagas en tu código fuente local se reflejará instantáneamente en el contenedor

- ``dockerfile: Dockerfile``: Especifica que el Dockerfile a usar para construir la imagen está en el mismo directorio y se llama Dockerfile. El dockerfilse sigue siendo relevante. 

- ``external: True``: Indica a Docker Compose que esta red(``platzinet``) ya existe en tu entorno Docker y no debe intentar crearla, para que no genere error.

Tambien modificaremos el *Dockerfile*:

```sh
FROM alpine:latest

RUN apk add --update python3

RUN apk add py3-pip

WORKDIR /web-server

COPY ["./requirements.txt", "."]

RUN pip install -r requirements.txt --break-system-packages

COPY [".", "."]

CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
```

-``COPY [".", "."]``: Copia todo el contenido restante de tu directorio de proyecto local (donde se encuentra el Dockerfile) al directorio de trabajo (``/web-server``) dentro del contenedor. Esto incluye main.py, datos.py, y cualquier otro archivo de tu aplicación, excepto los que excluyas con un ``.dockerignore``.

- ``CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]``: Define el comando predeterminado que se ejecutará cuando se inicie un contenedor a partir de esta imagen

- ``--port 8000``: Especifica que Uvicorn debe escuchar las conexiones en el puerto 8000 dentro del contenedor.

Nota que no estamos usando la opcion *--reload* asi que cada vez que halla un cambio en el codigo abra que buildiar una vez mas. 🚨 con:


### Levantando el servicio app-temp

Buildea con: `docker compose build`

Ejecuta el servicio en segundo plano con ``docker compose up -d``, y levanta el contenedor que contiene la BD de mongo con `docker run` como lo vienes haciendo. 

Abre otra terminal para ver los logs en tiempo real con `docker compose logs -f app-temp`, donde `app-temp` es el nombre del servicio definido en el `.yml`, y realiza un par de peticiones a cualquiera de los endpoints. Luego cierra el servicio con `docker compose down` en la misma terminal donde levantaste el servicio, en la terminal de logs veras: 

![](https://imgur.com/sO8AwWI.png)

Solo queria mostrarte que los eventos definidos con `on_event` tanto en `startup` como en `shutdown` estan funcionando.

#### Modificando para habilitar hot reload

``CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000", "--reload"]``

vuelve a buildiar

